# Project 4: West Nile Virus Analysis

## Part 2 - Modelling and Evaluation

In this notebook we are going to blah blah blah


Modelling 
- Summary of Modelling and Methodology

    Methodology: 
    - Base models with no tuning
    - Models using polyn
    
    
    


Evaluation
- Best Model
- Uploading to Kaggle 
- Recommendations and Conclusion

#### Contents

The contents of this notebook are as follows:
- [Imports](#Imports)

- [Modelling](#Modelling)

- [Evaluation](#Evaluation)

- [Cost-Benefit Analysis](#Cost--BenefitAnalysis)

- [Recommendations and Conclusion](#Recommendations-and-Conclusion)

## Imports

In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from xgboost import XGBClassifier

# from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline, make_pipeline

In [2]:
# Function to save kaggle submissions for WnvPresent probabilities
def get_kaggle_format(prediction_probs):
    # load the sample submission file
    sub = pd.read_csv('../datasets/sampleSubmission.csv')
    sub['WnvPresent'] = prediction_probs
    
    # Output filename for kaggle submission
    i = 0
    while os.path.exists(f"../datasets/kaggle_sub_{i}.csv"):
        i += 1
    filename = f"../datasets/kaggle_sub_{i}.csv"
    
    # Save the DataFrame to a file
    sub.to_csv(filename, index=False)
    print(f'File name: {filename}')

In [3]:
# import data
pd.set_option('display.max_columns', None)
train = pd.read_csv('../datasets/combined_cleaned.csv')
test = pd.read_csv('../datasets/test_cleaned.csv')

In [4]:
train

,date,latitude,longitude,wnv_present,species_ohe,tmax,tmin,tavg,depart,dew_point,wet_bulb,heat,cool,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed,daylight,r_humid,bc,br,dz,fg,fg+,fu,gr,hz,mi,ra,sn,sq,ts,vc,year,month,week,day
0,2007-05-29,41.731922,-87.677512,0,3.0,88.0,62.5,75.5,8.5,58.5,65.5,0.0,10.5,0.0,29.415,30.10,5.80,17.0,6.95,896.0,179.994268,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2007,5,22,1
1,2007-05-29,41.954690,-87.800991,0,2.0,88.0,62.5,75.5,8.5,58.5,65.5,0.0,10.5,0.0,29.415,30.10,5.80,17.0,6.95,896.0,179.994268,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2007,5,22,1
2,2007-05-29,41.974089,-87.824812,0,2.0,88.0,62.5,75.5,8.5,58.5,65.5,0.0,10.5,0.0,29.415,30.10,5.80,17.0,6.95,896.0,179.994268,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2007,5,22,1
3,2007-05-29,41.867108,-87.654224,0,2.0,88.0,62.5,75.5,8.5,58.5,65.5,0.0,10.5,0.0,29.415,30.10,5.80,17.0,6.95,896.0,179.994268,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2007,5,22,1
4,2007-05-29,41.919343,-87.694259,0,2.0,88.0,62.5,75.5,8.5,58.5,65.5,0.0,10.5,0.0,29.415,30.10,5.80,17.0,6.95,896.0,179.994268,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2007,5,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8423,2013-09-26,41.803423,-87.642984,0,1.0,75.0,52.5,64.0,2.0,52.0,58.0,1.0,0.0,0.0,29.370,30.04,3.95,8.5,4.40,719.0,153.933102,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2013,9,39,3
8424,2013-09-26,41.750498,-87.605294,0,1.0,75.0,52.5,64.0,2.0,52.0,58.0,1.0,0.0,0.0,29.370,30.04,3.95,8.5,4.40,719.0,153.933102,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2013,9,39,3
8425,2013-09-26,41.740641,-87.546587,0,1.0,75.0,52.5,64.0,2.0,52.0,58.0,1.0,0.0,0.0,29.370,30.04,3.95,8.5,4.40,719.0,153.933102,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2013,9,39,3
8426,2013-09-26,41.963976,-87.691810,0,1.0,75.0,52.5,64.0,2.0,52.0,58.0,1.0,0.0,0.0,29.370,30.04,3.95,8.5,4.40,719.0,153.933102,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2013,9,39,3


In [5]:
test

,date,latitude,longitude,species_ohe,tmax,tmin,tavg,depart,dew_point,wet_bulb,heat,cool,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed,daylight,r_humid,bc,br,dz,fg,fg+,fu,gr,hz,mi,ra,sn,sq,ts,vc,year,month,week,day
0,2008-06-11,41.954690,-87.800991,2.0,86.0,63.5,75.0,6.0,55.5,64.0,0.0,10.0,0.00,29.310,29.98,9.15,18.0,10.2,910.0,197.412262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,6,24,2
1,2008-06-11,41.954690,-87.800991,1.0,86.0,63.5,75.0,6.0,55.5,64.0,0.0,10.0,0.00,29.310,29.98,9.15,18.0,10.2,910.0,197.412262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,6,24,2
2,2008-06-11,41.954690,-87.800991,3.0,86.0,63.5,75.0,6.0,55.5,64.0,0.0,10.0,0.00,29.310,29.98,9.15,18.0,10.2,910.0,197.412262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,6,24,2
3,2008-06-11,41.954690,-87.800991,0.0,86.0,63.5,75.0,6.0,55.5,64.0,0.0,10.0,0.00,29.310,29.98,9.15,18.0,10.2,910.0,197.412262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,6,24,2
4,2008-06-11,41.954690,-87.800991,0.0,86.0,63.5,75.0,6.0,55.5,64.0,0.0,10.0,0.00,29.310,29.98,9.15,18.0,10.2,910.0,197.412262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,6,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116288,2014-10-02,41.925652,-87.633590,0.0,73.5,64.5,69.5,8.5,62.0,64.0,0.0,4.5,0.52,29.065,29.78,7.20,17.0,7.9,703.0,129.756173,0,1,0,0,0,0,0,0,0,1,0,0,1,0,2014,10,40,3
116289,2014-10-02,41.925652,-87.633590,0.0,73.5,64.5,69.5,8.5,62.0,64.0,0.0,4.5,0.52,29.065,29.78,7.20,17.0,7.9,703.0,129.756173,0,1,0,0,0,0,0,0,0,1,0,0,1,0,2014,10,40,3
116290,2014-10-02,41.925652,-87.633590,0.0,73.5,64.5,69.5,8.5,62.0,64.0,0.0,4.5,0.52,29.065,29.78,7.20,17.0,7.9,703.0,129.756173,0,1,0,0,0,0,0,0,0,1,0,0,1,0,2014,10,40,3
116291,2014-10-02,41.925652,-87.633590,0.0,73.5,64.5,69.5,8.5,62.0,64.0,0.0,4.5,0.52,29.065,29.78,7.20,17.0,7.9,703.0,129.756173,0,1,0,0,0,0,0,0,0,1,0,0,1,0,2014,10,40,3


The cleaned train and test datasets contain, 8428 and 116293 observations, respectively. We have **xx** features from the original data, combining variables from weather, location, species data etc. These are used to predict the target variable, probability if the west nile virus will be detected in a particular trap.

In [6]:
# check dtypes
train.dtypes

date             object
latitude        float64
longitude       float64
wnv_present       int64
species_ohe     float64
tmax            float64
tmin            float64
tavg            float64
depart          float64
dew_point       float64
wet_bulb        float64
heat            float64
cool            float64
precip_total    float64
stn_pressure    float64
sea_level       float64
result_speed    float64
result_dir      float64
avg_speed       float64
daylight        float64
r_humid         float64
bc                int64
br                int64
dz                int64
fg                int64
fg+               int64
fu                int64
gr                int64
hz                int64
mi                int64
ra                int64
sn                int64
sq                int64
ts                int64
vc                int64
year              int64
month             int64
week              int64
day               int64
dtype: object

In [7]:
# Get the training features and target variable
X = train.drop(columns=['date', 'wnv_present'])
y = train['wnv_present']

# Get the test dataset features
X_test = test.drop(columns=['date'])

In [8]:
# split the data into train and val for basic model building
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, shuffle=False)

In [9]:
y_train.value_counts(normalize=True)

0    0.9595
1    0.0405
Name: wnv_present, dtype: float64

## Modelling

Now we can build the models and train them using the training data. Since this is a classification problem of predicting whether a particular trap would have mosquitos with west nile virus or not, we will try some of them and run `GridSearchCV` to see which one gives us best results. Later, we will evaluate and discuss various metrics for the different models by interpreting the results.

### Pipelines

It is convenient to create pipelines using the `imblearns`'s `Pipeline` method, which is very similar to `sklearn's`, but we use this since we are using SMOTE. We can also write a function to get the results and print evaluations, so it is easier to consolidate everything when we run grid-search at once.

In [11]:
#dictionary for pipeline ============== SMOTE or SMOTENC
pipes_dict = {
    'pipe_bc': Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('bc', BaggingClassifier(random_state=42))
    ]),
    'pipe_et': Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('et', ExtraTreesClassifier(random_state=42))
    ]),
    'pipe_lr': Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('lr', LogisticRegression(random_state=42))
    ]),
    'pipe_rf': Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42))
    ]),
    'pipe_svc': Pipeline([
        ('ss', StandardScaler()),
        ('smote', SMOTE(random_state=42)),
        ('svc', SVC(random_state=42))
    ]),
    'pipe_xgb': Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('xgb', XGBClassifier())
    ])
}

In [15]:
#dictionary for pipeline parameters 
pipe_params = {
    'pipe_bc_params': {
        'bc__bootstrap_features': [True, False],
        'bc__n_estimators': [100, 500, 1000],
        'bc__max_features': [0.1, 0.5, 1.0]
    },
    'pipe_et_params': {
        'et__bootstrap': [True, False],
        'et__ccp_alpha': 0.0,
        'et__max_depth': [2, 3, 5]
    },
    'pipe_lr_params': {
        'lr__C': [1.0],
        'lr__max_iter': [100],
        'lr__l1_ratio': None
    },
    'pipe_rf_params': {
        'rf__max_depth': [2, 5, 10],
        'rf__max_features': ['sqrt', 'log2'],
        'rf__min_samples_leaf': [2, 5, 10],
        'rf__n_estimators': [100, 500, 1000],
#         'rf__warm_start': [True, False]
    },    
    'pipe_svc_params': {
        'svc__C': [1.0, 10.0, 100.0],
        'svc__degree': [2, 3, 5],
        'svc__kernel': ['rbf', 'poly', 'linear']
    },
    'pipe_xgb_params': {
        'xgb__booster': ['gbtree', 'gblinear', 'dart'],
        'xgb__max_depth': [2, 3, 5]
    }
}

#### Functions for modelling

In [20]:
#function for model
#list to store results and evaluation metrics for models
model_results = []

#store the models and params from gridsearch to dict
gs_dict = {}

#function to train model and store results in model_results list
def model_eval(pipeline, params):
    #for loop to check for knn model as knn model has StandardScaler in its pipeline
    for lists in pipeline.steps:
        if 'svc' in lists:
            model = pipeline.steps[2][0]   #model name
        else:
            model = pipeline.steps[1][0]
            
    gs = f'gs_{model}'
    
    gs_dict[gs] = GridSearchCV(estimator = pipeline,
                      param_grid = params,
                      scoring = 'roc_auc',
                      verbose = 1,
                      n_jobs = 4)

    gs_dict[gs].fit(X_train, y_train)  #fit our model onto gridsearchCV
    results = {}  #initialise results dictionary
    
    results['Classifier'] = model
    
    results['Train Accuracy Score'] = gs.score(X_train, y_train) #train acctracy score
    y_pred = gs_dict[gs].predict(X_val) #get the predictions
    results['Val Accuracy Score'] = accuracy_score(y_val, y_pred) #test accuracy score
    results['Recall'] = recall_score(y_val, y_pred)  #recall score
    results['Precision'] = precision_score(y_val, y_pred)  #precision score
    results['F1-Score'] = f1_score(y_val, y_pred) #f1-score
    results['ROC-AUC'] = roc_auc_score(y_val, y_pred) #roc-auc curve
    model_results.append(results) #append dictionary into model_results list
    
    return gs_dict[gs].best_estimator_, gs_dict[gs].best_params_

### Generating Models

#### Logistic Regression

In [15]:
# pipeline creation
pipe_lr = make_pipeline(SMOTE(random_state=42),
                        StandardScaler(), 
                        LogisticRegression())

# hyperparameters to be searched over
pipe_lr_params = {
    'logisticregression__penalty' : ['l2'], 
    'logisticregression__C' : [0.001], #0.0001,0.001,0.01,0.1,1,10,20
    'logisticregression__max_iter' : [1000] 
}

# gridsearching
gs_lr = GridSearchCV(pipe_lr,
                     param_grid=pipe_lr_params,
                     cv=5,
                     scoring='roc_auc')

# fit to train set
gs_lr.fit(X_train, y_train)

# predict on validation set
lr_pred = gs_lr.predict(X_val)

# best parameters and ROC AUC scores
print('Best parameters: ', gs_lr.best_params_)
print('Best CV ROC AUC score: ', gs_lr.best_score_)
print('Training ROC AUC score:', gs_lr.score(X_train, y_train))
print('Validation ROC AUC score:', gs_lr.score(X_val, y_val))

Best parameters:  {'logisticregression__C': 0.001, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}
Best CV ROC AUC score:  0.672153729270261
Training ROC AUC score: 0.8229261129431162
Validation ROC AUC score: 0.711442055584688


#### Decision Trees

In [18]:
# pipeline creation
pipe_dt = make_pipeline(SMOTE(random_state=42),
                              DecisionTreeClassifier())

# hyperparameters to be searched over
pipe_dt_params = {
    'decisiontreeclassifier__max_depth': [2], #2,4,10
    'decisiontreeclassifier__min_samples_split': [2], #2,5,10,15
    'decisiontreeclassifier__min_samples_leaf': [2], #2,4,6
    'decisiontreeclassifier__ccp_alpha': [0] #0,0.01,1
}

# gridsearching
gs_dt = GridSearchCV(pipe_dt,
                          param_grid=pipe_dt_params,
                          cv=5,
                          scoring='roc_auc')

# fit to train set
gs_dt.fit(X_train, y_train)

# # predict on validation set
dt_pred = gs_dt.predict(X_val)

# best parameters and ROC AUC scores
print('Best parameters: ', gs_dt.best_params_)
print('Best CV ROC AUC score: ', gs_dt.best_score_)
print('Training ROC AUC score:', gs_dt.score(X_train, y_train))
print('Validation ROC AUC score:', gs_dt.score(X_val, y_val))

Best parameters:  {'decisiontreeclassifier__ccp_alpha': 0, 'decisiontreeclassifier__max_depth': 2, 'decisiontreeclassifier__min_samples_leaf': 2, 'decisiontreeclassifier__min_samples_split': 2}
Best CV ROC AUC score:  0.6541125302933654
Training ROC AUC score: 0.7206850911995053
Validation ROC AUC score: 0.6842868379653907


#### AdaBoost

In [23]:
# pipeline creation
pipe_adaboost = make_pipeline(SMOTE(random_state=42),
                              AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),
                                                 random_state=42))

# hyperparameters to be searched over
pipe_adaboost_params = {
    'adaboostclassifier__learning_rate': [0.8], #0.7,0.8,0.9,1
    'adaboostclassifier__n_estimators': [25],  #20,25,50,100,200,300
    'adaboostclassifier__base_estimator__max_depth': [1] #1,2
}

# gridsearching
gs_adaboost = GridSearchCV(pipe_adaboost,
                          param_grid=pipe_adaboost_params,
                          cv=5,
                          scoring='roc_auc')

# fit to train set
gs_adaboost.fit(X_train, y_train)

# predict on validation set
adaboost_pred = gs_adaboost.predict(X_val)

# best parameters and ROC AUC scores
print('Best parameters: ', gs_adaboost.best_params_)
print('Best CV ROC AUC score: ', gs_adaboost.best_score_)
print('Training ROC AUC score:', gs_adaboost.score(X_train, y_train))
print('Validation ROC AUC score:', gs_adaboost.score(X_val, y_val))

Best parameters:  {'adaboostclassifier__base_estimator__max_depth': 1, 'adaboostclassifier__learning_rate': 0.8, 'adaboostclassifier__n_estimators': 25}
Best CV ROC AUC score:  0.738995845658956
Training ROC AUC score: 0.8686891359233306
Validation ROC AUC score: 0.736962506554798


#### Baseline Model

In [18]:
# Baseline results
y.value_counts(normalize=True)

0    0.945895
1    0.054105
Name: wnv_present, dtype: float64

As discussed in the EDA part that the classes are highly unbalanced and we will employ the SMOTE technique to make the two classes of same distribution. It is short for Synthetic Minority Oversampling TEchnique, which means we oversample the minority class (for training data) so that the two output classes are balanced. This avoids overtraining the model on one class and makes better predictions.

#### Models (No tuning)

#### Models (PCA)

#### Models (SMOTE)



## Evaluation

In [ ]:
# best model
best_model = gs_dict['gs_xxx']

In [ ]:
# write a function to plot the confusion matrix and ROC curve
def plot_cm_roc(model, X, y):
    roc_train = roc_auc_score(y_true=y_train, y_score=model.predict_proba(X_train)[:,1])
    roc_test = roc_auc_score(y_true=y, y_score=model.predict_proba(X)[:,1])
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,7))
    ax[0].set_title("Confusion Matrix for test data")
    ax[1].set_title("AUC ROC curve")
    plot_confusion_matrix(estimator=model,
                          X=X,
                          y_true=y,
                          cmap='Blues',
                          values_format='d',
                          colorbar=False,
                          ax=ax[0])
    plot_roc_curve(estimator=model, X=X_train, y=y_train, ax=ax[1])
    plot_roc_curve(estimator=model, X=X, y=y, ax=ax[1])
    ax[1].legend([f'train (AUC = {round(roc_train, 2)})', f'valid  (AUC = {round(roc_test, 2)})']);

In [ ]:
model_performance = pd.DataFrame(model_results).sort_values(by= 'ROC-AUC', ascending = False)
model_performance.reset_index(drop=True)

### Kaggle

In [ ]:
#save the kaggle submission file
get_kaggle_format(best_model.predict_proba(X_test))

## Cost-Benefit Analysis

## Recommendations and Conclusion